# The build cache


### Introduction

In the last lab, we wrote the Dockerfile to launch a flask application, created the image and then built the related container.  In this lesson, let's learn some steps towards better structuring that DockerFile.

### The Docker Cache

Let's imagine that our DockerFile from the last lesson looked something like the following:

```DockerFile
FROM python:3.7-alpine
WORKDIR /usr/src/app
COPY . .
RUN pip3 install -r requirements.txt
CMD ["flask", "run", "--host=0.0.0.0"]
```

Now, if we rebuild the image, let's pay close attention to the build process.  

`docker build -t jek2141/flask_lab .`

<img src="./flask_lab_build.png" width="100%">

Notice that with steps 2 through 5, before each step it says `CACHED`.  Essentially, in each step, Docker sees that it has already built this layer and therefore does not need to rebuild it.  Instead it just uses it's existing image, and then moves on from there.  

### Introducing a Problem

Ok, now let' change some the code in for our Flask app, and see how this effects the cache.

> We'll make a simple change by just `lisa simpson`, to go along with bart simpson.

```python
app = Flask(__name__)

@app.route('/users')
def index():
    return jsonify({'users': ['bart simpson', 'lisa simpson']})

if __name__ == '__main__':
    app.run(debug=True)
```

Ok, let's rebuild the image.

<img src="./busted_cache.png" width="100%">

Notice that this time, we do not see the cache used in steps 3 and 4.  So it makes that step 3 can no longer use the cache.  After all, we changed the codebase.  However, it's not ideal that ever subsequent layer also needs to be rebuilt.  This is particularly inefficient when it comes to the `requirements.txt` file.  Installing all of our packages may take a while to rebuild, and it would be nice to avoid needing to reinstall our package, every time we change our code.  

We can avoid this if we change our code to the following.

```DockerFile
FROM python:3.7-alpine

WORKDIR /usr/src/app

COPY ./requirements.txt .

RUN pip3 install -r requirements.txt

COPY . .
CMD ["flask", "run", "--host=0.0.0.0"]
```

This time, after changing the working directory, we first only move over the `requirements.txt` file, and then perform the install.  Afterwards, we move over the rest of the codebase.  This way, if we only change the codebase, we have not affected any of the the previous steps.  This allows us to reuse more of our previous layers.  Ok, let's put it to the test.

<img src="./cached_proper.png" width='70%'>

So as we can see, this time around steps, 2, 3 and 4 did rely on the cache, as they were not changed.  Only our codebase as the last step was changed.

### Summary

In this lesson, we learned about the Docker cache.  We saw that, when building an image Docker does not rebuild layers that already exist.  However, if one of the layers is changed, then every subsequent layer is also new, and thus needs to be rebuilt.  One of the more costly steps is in installing our pip packages.  To avoid a fresh install with every change to our codebase, we can first copy over and install the packages specified in the `requirements.txt` file, and then copy over the remaining files in our app.     

### Resources

* [Command vs Entrypoint](https://blog.codeship.com/understanding-dockers-cmd-and-entrypoint-instructions/)
* Docker image cache
* Docker Logs